Code majorly copied from https://github.com/tomasonjo/blogs/blob/master/llm/openaifunction_constructing_graph.ipynb
Author's medium article: https://medium.com/@bratanic-tomaz/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17

In [31]:
from dotenv import load_dotenv
import os

load_dotenv()
# OPENAI_API_KEY in .env file

True

In [7]:
from langchain.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username ="neo4j"
password = "athena_password"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [9]:
from typing import List, Dict, Any, Optional

from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument
)
from langchain.pydantic_v1 import Field, BaseModel
from langchain.schema import Document


class Property(BaseModel):
    """A single property consisting of key and value"""
    
    key: str = Field(..., description="key")
    value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(None, description="List of relationship properties")

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""

    nodes: List[Node] = Field(..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(..., description="List of relationships in the knowledge graph")

In [10]:
def format_property_key(s: str) -> str:
    words = s.split()

    if not words:
        return s

    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]

    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""

    properties = {}

    if not props:
        return properties

    for p in props:
        properties[format_property_key(p.key)] = p.value

    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""

    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()

    return BaseNode(id=node.id.title(), type=node.type.capitalize(), properties=properties)


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""

    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}

    return BaseRelationship(source=source, target=target, type=rel.type, properties=properties)

In [32]:
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [12]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]] = None
    ) -> None:

    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.run(document.page_content)

    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )

    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [26]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import TokenTextSplitter

markdown_path = "../../../../09-01-23 Readings.md"
loader = UnstructuredMarkdownLoader(markdown_path)
raw_docs = loader.load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

docs = text_splitter.split_documents(raw_docs)

In [28]:
docs

[Document(page_content="http://www.livescience.com/51984-ancient-spartan-palace-unearthed.html\n- Mycenaean Sparta - “Ayios Vassileios” - burnt to ground in 14th century bc - potentially lost Spartan palace\n- Linear B script tablets - kept unbaked for temporary use - baked by fire\n    - names, records, intricate bureaucracy\n    - these tablets would only be found in an administrative center in Mycenaean culture\n\nhttps://www.forbes.com/sites/kristinakillgrove/2015/08/26/the-most-interesting-thing-about-that-ancient-spartan-palace-isnt-the-palace-its-the-language/?sh=3a52208b478e\n- lots of imported & otherwise high value items that show the importance of the site\n- “Linear B is probably related to Linear A, which was used for writing down the Minoan language. \xa0Texts in both Linear A and Linear B were initially found at\xa0Knossos\xa0on Crete by the renowned archaeologist Sir Arthur Evans at the turn of the 20th century”\n- “When Linear B tablets were also found on the Greek mai

In [30]:
from tqdm import tqdm

for i, d in tqdm(enumerate(docs), total=len(docs)):
    extract_and_store_graph(d)

100%|██████████| 2/2 [04:10<00:00, 125.48s/it]
